
# imports

In [831]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from numpy.random import seed
seed(888)
tf.set_random_seed(404)
import os
from PIL import Image
from time import strftime

# Constants

In [832]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = "tensorboard_mnist_digit_logs/"
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT * IMAGE_WIDTH * CHANNELS

# Get The Data

In [833]:
%%time
y_train_all= np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 0 ns
Wall time: 12.5 ms


In [834]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 6.88 s
Wall time: 6.95 s


In [835]:
%%time
x_test_all = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: total: 1.11 s
Wall time: 1.25 s


In [836]:
%%time
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

CPU times: total: 0 ns
Wall time: 3.22 ms


# Explore

In [837]:
x_train_all.shape

(60000, 784)

In [838]:
y_train_all.shape

(60000,)

# Data PreProcessing

In [839]:
# Re-Scale
x_train_all, x_test_all = x_train_all/255.0, x_test_all/255.0

# Convert ttarget values to one-hot encoding:

In [840]:
y_train_all = np.eye(10)[y_train_all]
y_test = np.eye(10)[y_test]

In [841]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [842]:
X = tf.placeholder(tf.float32, [None, TOTAL_INPUTS], name="X")
Y = tf.placeholder(tf.float32, [None, NR_CLASSES], name='Y')


## Neural Network Architechture

### Hyperparameters

In [843]:
nr_epochs = 50
learning_rate = 0.001
n_hidden1 = 512
n_hidden2 = 64


In [844]:
# with tf.name_scope("first hidden layer"):
#     initial_w1 = tf.truncated_normal([TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_w1, name="w1")
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_b1, name='b1')
#
#     layer1_in =  tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [845]:
# with tf.name_scope('second hidden layer'):
#     initial_w2 = tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_w2, name="w2")
#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_b2, name="b2")
#     layer2_in =  tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [846]:
# with tf.name_scope('output layer'):
#     initial_w3 = tf.truncated_normal([n_hidden2, NR_CLASSES], stddev=0.1, seed=42) # the first element is the prev layer and the second is the current layer
#
#     w3 = tf.Variable(initial_w3, name="w3")
#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES]) # shape is the current layer
#     b3 = tf.Variable(initial_b3, name="b3")
#     layer3_in =  tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [847]:
def setup_layer(input, weight_dim,bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(weight_dim, stddev=0.1, seed=42) # the first element is the prev layer and the second is the current layer
        w = tf.Variable(initial_w, name="w")
        initial_b = tf.constant(value=0.0, shape=bias_dim) # shape is the current layer
        b = tf.Variable(initial_b, name="b")
        layer_in =  tf.matmul(input, w) + b
        if name == 'out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)

            tf.summary.histogram('weights', w)
            tf.summary.histogram('biases', b)
        return layer_out

In [848]:
# layer_1 = setup_layer(X, [TOTAL_INPUTS, n_hidden1], [n_hidden1], 'layer_1')
# layer_2 = setup_layer(layer_1, [n_hidden1, n_hidden2], [n_hidden2], 'layer_2')
# output = setup_layer(layer_2, [n_hidden2, NR_CLASSES], [NR_CLASSES], 'out')
#
# model_name = f'{n_hidden1}-{n_hidden2} LR {learning_rate} E{nr_epochs}'


In [849]:
layer_1 = setup_layer(X, [TOTAL_INPUTS, n_hidden1], [n_hidden1], 'layer_1')
layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8,name='dropout_layer')
layer_2 = setup_layer(layer_drop, [n_hidden1, n_hidden2], [n_hidden2], 'layer_2')
output = setup_layer(layer_2, [n_hidden2, NR_CLASSES], [NR_CLASSES], 'out')

model_name = f'{n_hidden1}-{n_hidden2} LR {learning_rate} E{nr_epochs}'


# TensorBoard Setup

In [850]:
invalid_chars = '/\:*?"<>|'  # Add any other invalid characters specific to your system
folder_name = f'{model_name} at {strftime("%H:%M")}'  # Use a valid separator like a hyphen (-)
for char in invalid_chars:
    folder_name = folder_name.replace(char, '_')

directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("worked!")

worked!


# Loss, Optimisation and Metrics

#### Defining Loss function

In [851]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining an optimiser

In [852]:
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [853]:
with tf.name_scope("accuracy_calc"):
    correct_prediction = tf.equal(tf.argmax(output,axis=1), tf.argmax(Y,axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [854]:
with tf.name_scope("performance"):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [855]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [856]:
sess = tf.Session()

#### Setup Filewriter and merge Summaries


In [857]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

# Initalise Variables

In [858]:
init = tf.global_variables_initializer()
sess.run(init)

# Batching the data

In [859]:
size_of_batch = 1000
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)
index_in_epoch = 0


In [860]:
def next_batch(batch_size, data, labels):

    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch

    return data[start:end], labels[start:end]

### Training loop

In [861]:
for epoch in range(nr_epochs):

    # ============= Training Dataset =========
    for i in range(nr_iterations):

        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)


    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)

    train_writer.add_summary(s, epoch)

    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')

    # ================== Validation ======================

    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.8619999885559082
Epoch 1 	| Training Accuracy = 0.8809999823570251
Epoch 2 	| Training Accuracy = 0.9200000166893005
Epoch 3 	| Training Accuracy = 0.9629999995231628
Epoch 4 	| Training Accuracy = 0.9739999771118164
Epoch 5 	| Training Accuracy = 0.9760000109672546
Epoch 6 	| Training Accuracy = 0.9769999980926514
Epoch 7 	| Training Accuracy = 0.9800000190734863
Epoch 8 	| Training Accuracy = 0.9800000190734863
Epoch 9 	| Training Accuracy = 0.9800000190734863
Epoch 10 	| Training Accuracy = 0.9810000061988831
Epoch 11 	| Training Accuracy = 0.984000027179718
Epoch 12 	| Training Accuracy = 0.9860000014305115
Epoch 13 	| Training Accuracy = 0.9850000143051147
Epoch 14 	| Training Accuracy = 0.9869999885559082
Epoch 15 	| Training Accuracy = 0.9890000224113464
Epoch 16 	| Training Accuracy = 0.9900000095367432
Epoch 17 	| Training Accuracy = 0.9860000014305115
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.99000000

# Make a prediction

In [862]:
img = Image.open('MNIST/test_img.png')
bw = img.convert("L")
image_array = np.invert(bw)



In [863]:
test_image = image_array.ravel()
test_image.shape


(784,)

In [864]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_image]})
print(f"prediction for test image is: {prediction}")

prediction for test image is: [2]


# Testing and Evaluation

In [866]:
test_accuracy = sess.run(accuracy, feed_dict={X:x_test_all, Y:y_test})
print(f" test accuracy of: {0:.2f}%".format(test_accuracy * 100))

 test accuracy of: 0.00%


# Reset for the next run

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()